In [ ]:
#PREPARES INDIVIDUAL CURVE DATA POINTS SKIPPING FIRST TWO LINES

import numpy as np
def data_prep(filename):

    data = np.genfromtxt(filename,delimiter=None,dtype=float,skip_header=2)
    return data
data=data_prep('curve-pi.txt')
data

array([[-1.570796,  0.      ],
       [-1.047198,  0.5     ],
       [ 0.      ,  1.      ],
       [ 1.047198,  0.5     ],
       [ 1.570796,  0.      ]])

In [ ]:
#DEFINES EVERY NEEDED SUMATORY FOR THE LINEAR QUATIONS SYSTEM

def sumatories(curve_data,degree):

    curve=data_prep(curve_data)
    x=curve[:,0]
    y=curve[:,1]
    n = len(curve[:,0])

    sums_x=[]
    sums_xy=[]

    for m in range((degree)*2+1): #runs over the possible exponents in each term of the equations (ascendant) (from 0 to 2m)

        sum_x=0
        sum_xy=0

        for i in range(len(x)): #run over datapoints performing the sum

            sum_x += curve[i,0]**(m)

            if m <= degree:
                sum_xy += curve[i,0]**(m) * curve[i,1]  # xy terms only run till m, unlike x terms

        sums_x.append(sum_x)

        if m <= degree:
            sums_xy.append(sum_xy)

    return sums_x, sums_xy #sums get accumulated in arrays to be indexed in the following function


In [ ]:
def linear_system_mat(curve_data,degree):

    M = np.zeros((degree+1,degree+2)) # 0-filled matrix of m+1,m+2 dimensions taking into account the equality (+2)

    sx,sxy = sumatories(curve_data,degree)

    h=0

    for row in range(degree+1): #runs over rows

            for col in range(degree+1): #runs over columns

                M[row,col] = sx[col+h]

            h += 1 #when a row is filled, the term to fill now is the first term plus the number of rows already filled

    for row in range(degree+1): #filling equality column

        M[row,degree+1] = sxy[row]

    return M



In [ ]:
def gaussjordan(curve_data,degree):

    M=linear_system_mat(curve_data,degree)
    rows, cols = M.shape
    coeff=[]

    for i in range(rows):

        pivot = np.argmax(np.abs(M[i:, i])) + i  # choosing pivot value
        M[[i, pivot]] = M[[pivot, i]]  # swapping columns to get the pivot in position 0,0
        div = M[i,i]
        M[i] /= div  # divides row by the diagonal term (pivot)
        div = div.astype(float)

        for j in range(rows):
            if j!=i:
                M[j,:] -= M[j,i] * M[i,:] #perfomrs operation over rows to make its values zero except for the diagonal

    for i in range(rows):

        result = M[i, cols - 1]/M[i,i] #solves equation
        coeff.append(result)

    return coeff


In [ ]:
def func(x,a):
    return a[0] + a[1]*x + a[2]*x**2 + a[3]*x**3 #+ a[4]*x**4# + a[5]*x**5

def fitting(curve_data,degree):

    import pandas as pd
    from IPython.display import display
    from IPython.display import Math
    import matplotlib.pyplot as plt

    #displays datapoints dataframe for clarity

    curve = data_prep(curve_data)
    x=curve[:,0]
    y=curve[:,1]
    curve2 = {'X':curve[:,0],'Y': curve[:,1]}
    curve_df = pd.DataFrame(curve2)
    display(curve_df)

    a = gaussjordan(curve_data,degree) #coefficients

    #displays general form of the polynomial

    function = r"y = a_{m} x^m + a_{m-1}x^{m-1} + ... + a_{2}x^{2} + a_{1}x^{1} +a_0"
    display(Math(function))

    #presents coefficients

    for i in range(degree+1):

        print(f"a{i} = {a[i]}")


    #function terms has to be defined according to polynomial degree

    #def func(x):
     #   return a[0] + a[1]*x + a[2]*x**2 #+ a[3]*x**3 #+ a[4]*x**4 + a[5]*x**5
    func(x,a)
    # plots datapoint and its fitting with the obtained parameters
    plt.scatter(curve_df['X'], curve_df['Y'],label='Data points')

    #generates x values for the fitting with polynomial in a generalizable range
    x_val_fit = np.linspace(curve[0,0], curve[-1,0], 1000)
    plt.xlim(curve[0,0], curve[-1,0])
    plt.xlabel("X")
    plt.ylabel("Y")

    plt.plot(x_val_fit,func(x_val_fit,a),'-r', label='Fitting')
    plt.legend()
    plt.show()

    print(func(np.pi/4,a))


In [ ]:
def compute_r_squared(curve_data,degree): #computes R2 value

    a = gaussjordan(curve_data,degree)
    curve=data_prep(curve_data)
    x=curve[:,0]
    y=curve[:,1]

    y_mean = np.mean(y)
    y_pred = np.polyval(a[::-1], x)
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - y_mean) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    #print(f"R2 = {r_squared}")
    return r_squared

In [ ]:
fitting('curve1.txt',3)
compute_r_squared('curve1.txt',3)



OSError: curve1.txt not found.

In [ ]:
print(np.cos(np.pi/4))


0.7071067811865476
